# Working with automated computations: Imported tables

Welcome back! In this session, we are going to continue working with the pipeline for the mouse calcium imaging example. 

In this session, we will learn to:

* import neuron imaging data from data files into an `Imported` table
* automatically trigger data importing and computations for all missing entries with `populate`

First thing first, let's import `datajoint` again.

In [ ]:
import datajoint as dj

As we are going to perform some computations, let's go ahead and import NumPy and Matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now we would like to continue working with the tables we defined in the previous notebook. To do so, we would need the classes for each table: `Mouse` and `Session`. We can either redefine it here, but for your convenience, we have included the schema and table class definitions in a package called `tutorial_pipeline.mouse_session`, from which you can import the classes as well as the schema object. We will use the schema object again to define more tables.

In [ ]:
from tutorial_pipeline.mouse_session import schema, Mouse, Session

Take a quick look at the tables Mouse and Session

In [ ]:
Mouse()

In [ ]:
Session()

The `mouse_session.py` also fills each table with data to make sure we are all on the same page.

## Define table `Scan` for meta information of each calcium imaging scan

Now let's define a table `Scan` that describes a scanning in an experimental session that stores the meta information of a particular scan.

In [ ]:
@schema
class Scan(dj.Manual):
    definition = """
    -> Session
    scan_idx    : int           # scan index
    ---
    depth       : float         # depth of this scan
    wavelength  : float         # wavelength used
    laser_power : float         # power of the laser used
    fps         : float         # frames per second
    file_name    : varchar(128) # name of the tif file
    """

This table is dependent on the table `Session`, inheriting its primary key attributes, with an additional primary key attribute `scan_idx`. One session could contain multiple scans, which is another example of **one-to-many** relationship. We could take a look at the Diagram again.

In [ ]:
dj.Diagram(schema)

The thin solid line connecting `Mouse`-`Session`, and `Session`-`Scan` indicates **one-to-many relationship**.  

The `____` indicates **additional primary key attribute(s)** apart from the ones inherited from its parents.

Here we have prepared two tif files of scanning in the `data` folder `example_scan_01.tif` and `example_scan_02.tif` 

In [ ]:
from tutorial_pipeline import data_dir
data_dir

In [ ]:
for f in data_dir.glob('*.tif'):
    print(f)

Let's insert these meta information manually.

In [ ]:
Scan.insert([
    {'mouse_id': 0, 'session_date': '2017-05-15', 'scan_idx': 1, 
     'depth': 150, 'wavelength': 920, 'laser_power': 26, 'fps': 15, 'file_name': 'example_scan_01.tif'},
    {'mouse_id': 0, 'session_date': '2017-05-15', 'scan_idx': 2, 
     'depth': 200, 'wavelength': 920, 'laser_power': 24, 'fps': 15, 'file_name': 'example_scan_02.tif'},
])

In [ ]:
Scan()

## Looking at the raw data

Let's first load one raw data and take a look at the data:

In [ ]:
import os
from skimage import io
im = io.imread(data_dir / 'example_scan_01.tif')
print(im.shape)

This tiff file contains 100 frames. Let's take the average of the images over frames and look at it.

In [ ]:
# ENTER YOUR CODE! - compute the avg frame with np.mean of axis=0
avg_image = np.mean(im, axis=0)
plt.imshow(avg_image, cmap=plt.cm.gray)

## Defining table for average fluorescence across frames

Now let's create a table `AverageFrame` to compute and save the average fluorescence. 

For each scan, we have one average frame. Therefore, the table shares the exact same primary key as the table `Scan`

In [ ]:
@schema
class AverageFrame(dj.Imported):
    definition = """
    -> Scan
    ---
    average_frame   : longblob     # average fluorescence across frames
    """

Let's check the state of our pipeline.

In [ ]:
dj.Diagram(schema)

Note that `AverageFrame` and `Scan` share the exact same primary key attributes, so `Scan` and `AverageFrame` have a **one-to-one** relationship, which is indicated with a thick solid line.

We defined `average_frame` as a `longblob` so that it can store a NumPy array. This NumPy array will be imported and computed from the file corresponding to each scan.

Note that our `AverageFrame` class inherits from `dj.Imported` instead of `dj.Manual` like others. This is because **this table's content will depend on data imported from an external file**. The `Manual` vs `Imported` are said to specify the **tier of the table**.

## DataJoint table tiers

In DataJoint, the tier of the table indicates **the nature of the data and the data source for the table**. So far we have encountered two table tiers: `Manual` and `Imported`, and we will encounter the two other major tiers in this session. 

DataJoint tables in `Manual` tier, or simply **Manual tables** indicate that its contents are **manually** entered by either experimenters or a recording system, and its content **do not depend on external data files or other tables**. This is the most basic table type you will encounter, especially as the tables at the beginning of the pipeline. In the Diagram, `Manual` tables are depicted by green rectangles.

On the other hand, **Imported tables** are understood to pull data (or *import* data) from external data files, and come equipped with functionalities to perform this importing process automatically, as we will see shortly! In the Diagram, `Imported` tables are depicted by blue ellipses.

In [ ]:
dj.Diagram(schema)

## Importing data into the `Imported` table

Rather than filling out the content of the table manually using `insert1` or `insert` methods, we are going to make use of the `make` and `populate` logic that comes with `Imported` tables to automatically figure out what needs to be imported and perform the import!

## `make` and `populate` methods

`Imported` table comes with a special method called `populate`. Let's try calling it.

In [ ]:
# ENTER YOUR CODE! - call `populate` on the table
AverageFrame.populate()

Notice that `populate` call complained that a method called `make` is not implemented. Let me show a simple `make` method that will help elucidate what this is all about.

In [ ]:
@schema
class AverageFrame(dj.Imported):
    definition = """
    -> Scan
    ---
    average_frame   : longblob     # average fluorescence across frames
    """
    def make(self, key):  # key is the primary key of one of the entries in the table `Scan`
        print('key is', key)

Now, let's call `populate` again!

In [ ]:
# ENTER YOUR CODE! - call `populate` on the table
AverageFrame.populate()

When you call `populate` on an `Imported` table, this triggers DataJoint to look up all tables that the `Imported` table depends on.

For **every unique combination of entries in the depended or "parent" tables**, DataJoint calls `make` function, passing in the primary key of the parent(s).

Because `AverageFrame` depends on `Scan`, `AverageFrame`'s `make` method was called for each entry of `Scan`

In [ ]:
Scan()

Note that `make` only receives the *primary key attributes* of `Scan` (`mouse_id`, `session_date`, `scan_idx`) but not the other attributes.

## Implementing `make`

Now we have a better understanding of `make`, let's implement `make` to perform the importing of data from file.

In [ ]:
import os
@schema
class AverageFrame(dj.Imported):
    definition = """
    -> Scan
    ---
    average_frame   : longblob     # average fluorescence across frames
    """
    def make(self, key):  # key is the primary key of one of the entries in the table `Scan`
        # fetch data directory from table Session
        data_path = (Session & key).fetch1('data_path')
        
        # fetch data file name from table Scan
        file_name = (Scan & key).fetch1('file_name')
        
        # load the file
        im = io.imread(os.path.join(data_path, file_name))
        # compute the average image across the frames
        avg_image = np.mean(im, axis=0)
        
        # Now prepare the entry as a dictionary with all fields defined in the table.
        key['average_frame'] = avg_image # inherit the primary key from the table Scan
        
        # insert entry with the method `insert1()`
        self.insert1(key)
        
        print('\tPopulated Scan {mouse_id} - {session_date} - {scan_idx}'.format(**key))

Notice that we added the missing attribute information `average_frame` into the `key` dictionary, and finally **inserted the entry** into `self` = `AverageFrame` table. The `make` method's job is to create and insert a new entry corresponding to the `key` into this table!

Finally, let's go ahead and call `populate` to actually populate the `AverageFrame` table, filling it with data loaded and computed from data files!

In [ ]:
AverageFrame.populate()

Let's take a look at the table now.

In [ ]:
AverageFrame()

What happens if we call `AverageFrame.populate` again?

In [ ]:
AverageFrame.populate()

That's right - nothing! This makes sense, because we have computed `AverageFrame` for all entries in `Scan` and nothing is left to be computed.

Now what happens if we insert a new entry into `Scan`?

In [ ]:
Scan.insert1({
    'mouse_id': 100,
    'session_date': '2017-05-25',
    'scan_idx': 1,
    'depth': 150,
    'wavelength': 920,
    'laser_power': 25,
    'fps': 15,
    'file_name': 'example_scan_03.tif'
})

We can find all `Scan` without corresponding `AverageFrame` entry with the **negative restriction operator** `-`

In [ ]:
# select all Scan entries *without* a corresponding entry in AverageFrame
Scan - AverageFrame

In [ ]:
AverageFrame.populate()

`populate` found the uncomputed entry and processed it!

In [ ]:
AverageFrame()

Now instead of loading from the raw tif file, we are able fetch the average fluorescence image from this table.

In [ ]:
key = dict(mouse_id=0, session_date='2017-05-15', scan_idx=1)
avg_image = (AverageFrame & key).fetch1('average_frame')
plt.imshow(avg_image, cmap=plt.cm.gray)

## Summary

Congratulations! You have successfully extended your pipeline with a table to represent processed data (`AverageFrame` as `Imported` table), learned and implemented the `make()` and `populate()` call to load external data to your tables.

In [ ]:
dj.Diagram(schema)

At this point, our pipeline contains the core elements with data populated, ready for further downstream analysis.

In the next [session](./03-Computed%20Table,%20Lookup%20Table,%20and%20Part%20Table%20-%20Interactive.ipynb), we are going to introduce the concept of `Computed` table, and `Lookup` table, as well as learning to set up a automated computation routine.